# MNE : From raw data to epochs and evoked responses (ERF/ERP)

`
Authors:
Alexandre Gramfort
Denis A. Engemann
Jona Sassenhagen
`

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

First, load the mne package:

In [ ]:
import mne

We set the log-level to 'warning' so the output is less verbose

In [ ]:
mne.set_log_level('warning')

### Remember if you need help just ask... the machine

In [ ]:
mne.pick_types?

## Access raw data

Now we import the sample dataset.

You should already have it but if you don't it will be downloaded automatically (but be patient approx. 2GB)

In [ ]:
from mne.datasets import sample
# also explore other datasets, e.g. mne.datasets.XXX
data_path = sample.data_path()
# data_path = '/Users/alex/mne_data/MNE-sample-data'  # change to location of the data or use line above to download automatically

raw_fname = data_path + '/MEG/sample/sample_audvis_filt-0-40_raw.fif'

In [ ]:
print(raw_fname)

Read data from file:

In [ ]:
mne.io.read_raw_fif?

In [ ]:
raw = mne.io.read_raw_fif(raw_fname, preload=False)
print(raw)

Note the `preload=False` which states that no data is actually in memory.

For more details about IO of different file formats see [IO with MNE](http://martinos.org/mne/dev/manual/io.html)

Now let's look at the measurement info. It will give details about:

   - sampling rate
   - filtering parameters
   - available channel types
   - bad channels
   - etc.

In [ ]:
print(raw.info)

raw.info is just a dictionary:

In [ ]:
isinstance(raw.info, dict)

So we can access its elements this way:

In [ ]:
raw.info['sfreq']  # Sampling frequency

In [ ]:
raw.info['bads']  # list of marked bad channels

Next let's see what channels are present. It is available via the `raw.ch_names` attribute.

In [ ]:
raw.ch_names

You can index it as a list

In [ ]:
raw.ch_names[42]

In [ ]:
raw.ch_names[:10]

Channel type of a specific channel

In [ ]:
channel_type = mne.io.pick.channel_type(raw.info, 75)
print('Channel #75 is of type:', channel_type)

channel_type = mne.io.pick.channel_type(raw.info, 320)
print('Channel #320 is of type:', channel_type)

Info contains all the details about the sensors (type, locations, coordinate frame etc.)

In [ ]:
len(raw.info['chs'])

In [ ]:
raw.info['chs'][0]

In [ ]:
raw.info['chs'][330]

In [ ]:
raw.plot_sensors(kind='topomap', ch_type='grad');

## Accessing the data

To access the data just use the [] syntax as to access any element of a list, dict etc.

In [ ]:
start, stop = 0, 10
data, times = raw[:, start:stop]  # fetch all channels and the first 10 time points
print(data.shape)
print(times.shape)

In [ ]:
times

Note that it returns both the data and the times array.

# Visualizing raw data

See http://martinos.org/mne/stable/auto_tutorials/plot_visualize_raw.html
for more details.

Let's look at how to:
- browse data
- turn On/Off the PCA/SSP projections
- mark bad segments to obtained annotations
- group channel by types
- group channel by location

In [ ]:
%matplotlib qt

raw.plot()

In [ ]:
raw.annotations.onset

In [ ]:
raw.annotations.duration

In [ ]:
raw.annotations.description

In [ ]:
raw.annotations.save('raw_sample_annot.csv')

In [ ]:
!cat raw_sample_annot.csv

Save a segment of 150s of raw data (MEG only):

In [ ]:
picks_meg = mne.pick_types(raw.info, meg=True, eeg=False, eog=False,
                           stim=False, exclude='bads')
raw.save('sample_audvis_meg_raw.fif', tmin=0., tmax=150., picks=picks_meg, overwrite=True)

### Filtering

In [ ]:
raw_beta = mne.io.read_raw_fif(raw_fname, preload=True)

In [ ]:
raw_beta.filter(13, 30)

In [ ]:
raw_beta.filter?

In [ ]:
raw_beta = mne.io.read_raw_fif(raw_fname, preload=True)  # reload data with preload for filtering

# keep beta band
raw_orig = raw_beta.copy()
raw_beta.filter(13.0, 30.0)

# save the result
raw_beta.save('sample_audvis_beta_raw.fif', overwrite=True)

print(raw_beta.info)  # note the update of raw.info['lowpass'] and raw.info['highpass']

<div class="alert alert-success">
    <b>Exercise</b>:
     <ul>
    <li>Filter the raw data between 1Hz and 40Hz. Observe the absence of the slow drifts.</li>
    </ul>
</div>

In [ ]:
 # TODO

%matplotlib qt


<div class="alert alert-success">
    <b>Exercise</b>:
     <ul>
    <li>Plot the 10 first seconds of stimutation channel just using matplotlib.</li>
    </ul>
</div>

Tips:

- Pick the stim channel using `mne.pick_types`
- Get the data for this channel
- Plot it using `plt.plot`

In [ ]:
 # TODO

In [ ]:
%matplotlib inline


## Define and read epochs

First extract events:

In [ ]:
events = mne.find_events(raw, stim_channel='STI 014', verbose=True)

In [ ]:
events.shape

In [ ]:
type(events)

In [ ]:
# See parameters to fine tune event detection, e.g. consecutive non-zero
# mne.find_events?

In [ ]:
print(events[:5])  # events is a 2d array, (time, previous, trigger)

In [ ]:
events[events[:, 2] == 2]

In [ ]:
len(events[events[:, 2] == 2])

In [ ]:
len(events)

Let's visualize the paradigm:

In [ ]:
%matplotlib inline

fig = mne.viz.plot_events(events, raw.info['sfreq']);

Events are stored as 2D numpy array where the first column is the time instant and the last one is the event number. It is therefore easy to manipulate.

In [ ]:
events_new = events.copy()
events_new[events_new[:, 2] == 2, 2] = 1  # MATLAB- and R-like syntax
events_new[events_new[:, 2] == 4, 2] = 3
print(events_new[:5])
print(events[:5])

In [ ]:
events = events[events[:, 2] < 5]
fig = mne.viz.plot_events(events, raw.info['sfreq']);

For event trigger and conditions we use a Python dictionary with keys that contain "/" for grouping sub-conditions

In [ ]:
event_id = {"visual/left": 3, "visual/right": 4, "auditory/left": 1, "auditory/right": 2}

In [ ]:
fig = mne.viz.plot_events(events, raw.info['sfreq'], event_id=event_id);

In [ ]:
%matplotlib qt

raw.plot(event_id=event_id, events=events)

Define epochs parameters:

In [ ]:
tmin = -0.2  # start of each epoch (200ms before the trigger)
tmax = 0.5  # end of each epoch (500ms after the trigger)

Define the baseline period:

In [ ]:
baseline = (None, 0)  # means from the first instant to t = 0

Define peak-to-peak (amplitude range) rejection parameters for gradiometers, magnetometers and EOG:

In [ ]:
reject = dict(grad=4000e-13, mag=4e-12, eog=150e-6)  # this can be highly data dependent

<div class="alert alert-info">
    <b>REMARK</b>:
     <ul>
    <li>The <a href="https://autoreject.github.io/">autoreject</a> project aims to solve this problem of reject parameter setting. See the <a href="https://www.sciencedirect.com/science/article/pii/S1053811917305013">paper</a>.</li>
    </ul>
</div>

In [ ]:
# we are picky again, this time with EOG
picks_meg = mne.pick_types(raw.info, meg=True, eeg=False, eog=True,
                           stim=False, exclude='bads')
# we start by looking at magnetometer and gradiometer signals

Extract epochs:

In [ ]:
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=True,
                    picks=picks_meg, baseline=baseline, reject=reject)

In [ ]:
print(epochs)

In [ ]:
epochs.drop_bad()  # remove bad epochs based on reject

In [ ]:
epochs.load_data()  # load data in memory

Explore the epochs namespace

Hit ``epochs.<TAB>``

In [ ]:
epochs.

See how epochs were dropped

In [ ]:
epochs.plot_drop_log();

### Visualization Epochs

See [this page](https://mne.tools/stable/auto_tutorials/epochs/plot_visualize_epochs.html) for options on how to visualize epochs.

Here is just an illustration to make a so-called ERP/ERF image:

In [ ]:
%matplotlib inline
epochs.plot_image(picks=278, cmap='interactive', sigma=1., vmin=-250, vmax=250)

In [ ]:
%matplotlib qt

epochs.plot(block=True)  # Google Summer of code 2015 with Jaakko Leppäkangas

In [ ]:
mne.preprocessing.maxwell_filter?

### The epochs object is your MNE swiss army knife for processing segmented data!

- specialized methods for diagnostic plotting of data
- averaging
- saving
- manipulating data, e.g., rearranging or deleting single trials, resampling

## More diagnostic plotting

In [ ]:
%matplotlib inline
# epochs.drop_bad()
epochs.plot_drop_log();

In [ ]:
for drop_log in epochs.drop_log[:20]:
    print(drop_log)

In [ ]:
epochs.copy().drop(10, reason="I don't like this one").plot_drop_log();

## basic IO 

The standard scenario is saving the epochs into .fif file together with all the header data.

In [ ]:
epochs.save('sample-epo.fif')  # note that epochs are save in files ending with -epo.fif

In [ ]:
data = epochs.get_data()
data.shape

Scipy also supports reading and writing of matlab files. You can save your single trials with:

In [ ]:
from scipy import io
epochs_data = epochs.get_data()
print(epochs_data.shape)
io.savemat('epochs_data.mat', dict(epochs_data=epochs_data),
           oned_as='row')

### Average the epochs to get ERF/ERP and plot it!

In [ ]:
evoked = epochs.average()
print(evoked)

In [ ]:
%matplotlib qt

evoked.plot();

We can also show sensor position as line color:

In [ ]:
epochs

In [ ]:
%matplotlib inline
evoked.plot(spatial_colors=True);  # note the legend

In [ ]:
epochs['auditory/left'].average().plot(spatial_colors=True);  # note the legend

In [ ]:
raw.plot_sensors(ch_type='eeg');

In [ ]:
evoked.plot_topomap(ch_type='mag', times=[0.05, 0.1, 0.15]);

In [ ]:
evoked.plot_topomap(ch_type='grad', times=[0.05, 0.1, 0.15]);

In [ ]:
# pure topography plots called topomap in the MNE jargon
for ch_type in ('mag', 'grad'):
    evoked.plot_topomap(times=np.linspace(0.05, 0.15, 10), ch_type=ch_type);

Topoplot and time series can also be shown in one single plot:

In [ ]:
evoked.plot_joint();

In [ ]:
evoked.plot_joint(times=[0.1]);

## Accessing and indexing epochs by condition

Epochs can be indexed by integers or slices to select a subset of epochs but also with strings to select by conditions `epochs[condition]`

In [ ]:
epochs[0]  # first epoch

In [ ]:
epochs[:10]  # first 10 epochs

In [ ]:
epochs['visual/left']  # epochs for the left visual condition

In [ ]:
# remember ...
event_id

In event_id, "/" selects conditions in a hierarchical way, e.g. here, "auditory" vs. "visual", "left" vs. "right", and MNE can select them individually

In [ ]:
epochs['auditory/left'].average().\
    pick_types(meg='grad').crop(None, 0.2).plot(spatial_colors=True)

In [ ]:
event_id

In [ ]:
epochs['visual']  # epochs for the visual condition (either left or right)

In [ ]:
epochs['left']

To access the data of some epochs use the get_data method.

In [ ]:
epochs_data = epochs.get_data()
type(epochs_data), epochs_data.shape

`epochs_data` is a 3D array of dimension (239 epochs, 306 channels, 106 time instants).

Apply this to visualize all the conditions in `event_id`

In [ ]:
for condition in event_id:
    epochs[condition].average().plot_topomap(times=[0.1, 0.15], title=condition);

## Write evoked data to disk

In [ ]:
evoked.save('sample-ave.fif')  # note that the file for evoked ends with -ave.fif

or to write multiple conditions in 1 file

In [ ]:
evokeds_list = [epochs[k].average() for k in event_id]  # get evokeds
mne.write_evokeds('sample-ave.fif', evokeds_list)

### Reading evoked from disk

It is also possible to read evoked data stored in a fif file:

In [ ]:
evoked_fname = data_path + '/MEG/sample/sample_audvis-ave.fif'
evoked1 = mne.read_evokeds(evoked_fname, condition=0, baseline=(None, 0), proj=True)

Or another one stored in the same file:

In [ ]:
evoked2 = mne.read_evokeds(evoked_fname, condition=1,
                           baseline=(None, 0), proj=True)

Or give the explicit name of the averaged condition:

In [ ]:
evoked3 = mne.read_evokeds(evoked_fname, condition='Left Auditory',
                           baseline=(None, 0), proj=True)

**Remark:** Did you notice that you can apply some preprocessing on reading the evokeds from disk?

### Compute a contrast:

In [ ]:
contrast = mne.combine_evoked([evoked1, evoked2], [0.5, -0.5])

Note that this combines evokeds taking into account the number of averaged epochs (to scale the noise variance)

In [ ]:
print(evoked1.nave)  # average of 55 epochs
print(contrast.nave)  # average of 116 epochs

In [ ]:
print(contrast)

In [ ]:
fig = contrast.plot_joint()

### Save your figure as pdf

In [ ]:
%matplotlib qt
import numpy as np
contrast.plot_topomap(times=np.linspace(0.05, 0.15, 5), ch_type='mag')
plt.savefig('toto.pdf')
!open toto.pdf  # works only on a mac

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>Extract Epochs restricted to magnetometers on unfiltered data (sample_audvis_raw.fif)</li>
    <li>Construct epochs with a whole-epoch baseline. Then, filter raw data at 1 Hz, construct epochs from that. Compare the resulting Evokeds (filter vs. baseline)</li>
    <li>Plot the difference between all *visual* and all *auditory* stimulus presentations</li>
    <li>Recompute everything for EEG</li>
    </ul>
</div>

In [ ]:
%matplotlib qt
evoked.plot()

## ADVANCED: Some Python + MNE Kung Fu to plot selected channels and conditions

In [ ]:
%matplotlib inline

sensor = "MEG 1312"

evokeds = {k:epochs[k].average() for k in event_id}  # funny expression, no? Google "dict comprehension"

for condition, evoked in evokeds.items():  # that's what dictionaries are good for, looping!
    data = evoked.copy().pick_channels([sensor]).data[0]
    plt.plot(evoked.times, data * 1e13, label=condition)

plt.legend(loc="lower left")
plt.ylabel("fT/m")
plt.xlabel("time (s)")
plt.suptitle("MEG at electrode {}".format(sensor));

## ADVANCED: Customize your plots

Want to have every text in blue?

In [ ]:
fig = evoked.plot(show=False)  # butterfly plots
fig.subplots_adjust(hspace=1.0)
for text in fig.findobj(mpl.text.Text):
    text.set_fontsize(8)
    text.set_color('blue')
for ax in fig.get_axes():
    ax.axvline(0., color='red')
fig.savefig('plot_erf.pdf');